In [1]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(1, '../')
sys.path.insert(1, '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src')
print(sys.path)

['/home/avijendra/nl2sql_lib/nl2sql-generic/notebooks', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '../', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/avijendra/fserv/lib/python3.11/site-packages']


In [2]:
# Uncomment the below commands if required to re-install the libraries
# %pip install faiss-cpu --quiet
# %pip install google.cloud --quiet
# %pip install cloud-sql-python-connector --quiet
# %pip install pg8000 --quiet
# %pip install langchain-google-vertexai

In [3]:
import numpy as np
import csv
import faiss
from pandas import DataFrame
from datetime import datetime
from tqdm import tqdm

In [4]:
# PROJECT_ID = ! gcloud config get project
from nl2sql_generic import Nl2sqlBq
PROJECT_ID = 'sl-test-project-363109'
LOCATION = 'us-central1'
DATASET_ID = 'zoominfo'

# Query and Embeddings in PostgreSQL

## Initialise

In [5]:
from nl2sql_src.nl2sql_query_embeddings import PgSqlEmb

# Make sure you update the following variables in-line with your project
PGPROJ = "sl-test-project-353312"
PGLOCATION = 'us-central1'
PGINSTANCE = "test-nl2sql"
PGDB = "test-db"
PGTABLE = 'myqueries'
PGUSER = "postgres"
PGPWD = "test-nl2sql"
# INDX_FILE = 'saved_index_pgdata' # optoinal, considered as default

pge = PgSqlEmb(PGPROJ, PGLOCATION, PGINSTANCE, PGDB, PGUSER, PGPWD, pg_table=PGTABLE)
# mytable=PGTABLE

PGSQL - class Initiated


## Create Table

In [6]:
# Create Table in PostgresDB if not already existing
pge.create_table()

## Insert records to PostgreSQL and update index

In [16]:
examples = []
with open('examples.csv', 'r') as f:
    samples_data = csv.reader(f)
    header = next(samples_data)
    for r in samples_data:
        examples.append(r)

for example in tqdm(examples):
    pge.insert_row(example[0], example[1])

100%|██████████| 6/6 [00:16<00:00,  2.72s/it]


In [17]:
new_question = "How many employees in California are from Livingston County?"
mapped_sql = "select count(*) from `sl-test-project-363109.zoominfo.zi-retail-companies-headquartered-california` where ZI_C_COMPANY_COUNTY = 'Livingston County'"
pge.insert_row(new_question, mapped_sql)

## Retrieve data 

In [18]:
# Retrieve the data from the PostgreDB and return a dataframe
def extract_data():
    tmp = pge.extract_data()
    df = DataFrame(tmp.fetchall())
    print(df)
    return df

df = extract_data()
df

   q_id                                           question  \
0    45  How many marketing agents work from zip code 3...   
1    46  what is the revenue generated by sales agents ...   
2    47  What is the total number of employes providing...   
3    48  What is the total revenue for constuction indu...   
4    49  What are the top 5 industries in terms of reve...   
5    50  How many employees in California are from Dall...   
6    51  How many employees in California are from Livi...   

                                                 sql  \
0  select count(*) from `sl-test-project-363109.z...   
1  select sum(ZI_C_REVENUE) as rev from `sl-test-...   
2  select count(*) from `sl-test-project-363109.z...   
3  select sum(ZI_C_REVENUE) from `zoominfo.zi-com...   
4                       select ZI_C_INDUSTRY_PRIMARY   
5  select count(*) from `sl-test-project-363109.z...   
6  select count(*) from `sl-test-project-363109.z...   

                                     query_embedding  

,q_id,question,sql,query_embedding
0,45,How many marketing agents work from zip code 3...,select count(*) from `sl-test-project-363109.z...,"[0.04187563434243202, -0.0557498037815094, 0.0..."
1,46,what is the revenue generated by sales agents ...,select sum(ZI_C_REVENUE) as rev from `sl-test-...,"[-0.005688739009201527, -0.06865667551755905, ..."
2,47,What is the total number of employes providing...,select count(*) from `sl-test-project-363109.z...,"[0.06246837228536606, -0.04506361484527588, 0...."
3,48,What is the total revenue for constuction indu...,select sum(ZI_C_REVENUE) from `zoominfo.zi-com...,"[0.029781630262732506, -0.03381987288594246, -..."
4,49,What are the top 5 industries in terms of reve...,select ZI_C_INDUSTRY_PRIMARY,"[0.035186998546123505, -0.03914893791079521, 0..."
5,50,How many employees in California are from Dall...,select count(*) from `sl-test-project-363109.z...,"[0.035889022052288055, -0.03150845319032669, -..."
6,51,How many employees in California are from Livi...,select count(*) from `sl-test-project-363109.z...,"[-0.0085710184648633, -0.041441068053245544, -..."


## Search the VectorDB for closest matching queries

In [19]:
# Search for the closest matching queries
new_query = 'What is the total revenue for Hospitality industry'
output_json = pge.search_matching_queries(new_query)
output_json

Trying to read the index file ../../nl2sql-generic/nl2sql_src/cache_metadata/saved_index_pgdata


[{'question': 'What is the total revenue for constuction industry',
  'sql': 'select sum(ZI_C_REVENUE) from `zoominfo.zi-companies-under-1000-employees-offering-tuition` where ZI_C_INDUSTRY_PRIMARY = "Construction"  ;'},
 {'question': 'what is the revenue generated by sales agents for Manufacturing industry?',
  'sql': "select sum(ZI_C_REVENUE) as rev from `sl-test-project-363109.zoominfo.zi-companies-with-most-marketing-professionals` where ZI_C_INDUSTRY_PRIMARY = 'Manufacturing';"},
 {'question': 'What are the top 5 industries in terms of revenue for companies having headquarters in California ?',
  'sql': 'select ZI_C_INDUSTRY_PRIMARY'}]

## Clear VectorDB and re-create VectorDB from all data in PostgreSQL

In [11]:
pge.recreate_vectordb_index()

Number of elements :  28


## Clear all Data in DB

In [14]:
pge.empty_table()
df = extract_data()
df

Empty DataFrame
Columns: []
Index: []


""
